# LINEAR REGRESSION

In [1]:
import sys
sys.path.insert(0, '/home/anish/ASC_ML')

## Input and Read

In [3]:
from ASC_ML import dataframe_extractor as de

#### Example Use Case

##### For Directory path

In [4]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["MSSubClass", "LotFrontage"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading train.csv and test.csv of directory /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/
Train Dataset X Columns


,Id,MSZoning,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1.0,RL,8450.0,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,...,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal,208500.0
1,2.0,RL,9600.0,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,...,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal,181500.0
2,3.0,RL,11250.0,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,...,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal,223500.0
3,4.0,RL,9550.0,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,...,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml,140000.0
4,5.0,RL,14260.0,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,...,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal,250000.0


In [5]:
print("Train Dataset Y Columns")
dataset_list[1].head()

Train Dataset Y Columns


,MSSubClass,LotFrontage
0,60.0,65.0
1,20.0,80.0
2,60.0,68.0
3,70.0,60.0
4,60.0,84.0


In [7]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00383/risk_factors_cervical_cancer.csv'

# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from https://archive.ics.uci.edu/ml/machine-learning-databases/00383/risk_factors_cervical_cancer.csv
Train Dataset X Columns


,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18.0,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34.0,1.0,?,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,?,?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52.0,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,?,?,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,46.0,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,?,?,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
import requests

url="https://www.kaggle.com/louise2001/quantum-physics-articles-on-arxiv-1994-to-2009/download"

r = requests.get(url)

print(r.headers)

# directory1 = 'https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv'

# # returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(r.headers).df_list

# print("Train Dataset X Columns")
# dataset_list[0].head()

{'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 11 May 2022 02:50:36 GMT', 'Cache-Control': 'no-cache, no-store', 'Content-Encoding': 'br', 'Pragma': 'no-cache', 'Set-Cookie': 'CSRF-TOKEN=CfDJ8GHvYX0y1GNKpYA6n0D2CJYZdpR31GfuPenglQRkQ-Ovtk3WFsCzCyBmucf5hlooHcmQ5T1JIDqLUOdlhUE-2pSZEaxbKfy8OOoI0hyv4A; path=/; secure; samesite=lax; httponly, XSRF-TOKEN=CfDJ8GHvYX0y1GNKpYA6n0D2CJYvTStMYWZaLBvQTlaP6IlwAIhMCD3GjEc4jFXwYRERK1rZxVukQed_z0G6K5RXXSXcwqXtTvTvMOOi77gsToOLxw; path=/; secure; samesite=lax, CLIENT-TOKEN=eyJhbGciOiJub25lIiwidHlwIjoiSldUIn0.eyJpc3MiOiJrYWdnbGUiLCJhdWQiOiJjbGllbnQiLCJzdWIiOm51bGwsIm5idCI6IjIwMjItMDUtMTFUMDI6NTA6MzYuNTQwNjQ1OFoiLCJpYXQiOiIyMDIyLTA1LTExVDAyOjUwOjM2LjU0MDY0NThaIiwianRpIjoiMWNkNDFjMzEtNzE2My00OGRjLThmNDUtMGY0YjM2MTJjNDRjIiwiZXhwIjoiMjAyMi0wNi0xMVQwMjo1MDozNi41NDA2NDU4WiIsImFub24iOnRydWUsImZmIjpbIktlcm5lbHNTYXZlVG9HaXRIdWIiLCJHY2xvdWRLZXJuZWxJbnRlZyIsIktlcm5lbEVkaXRvcktpdHR5TW9kZSIsIkNhaXBFeHBvcnQiLCJDYWlwTnVkZ2UiLCJLZXJuZWxzRmlyZWJhc2VMb25nUG9sbGl

AttributeError: 'CaseInsensitiveDict' object has no attribute 'endswith'

##### For direct csv path

In [5]:
#For Weather Prediction dataset https://www.kaggle.com/budincsevity/szeged-weather
directory2 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/weather-history-dataset/weatherHistory.csv"

dataset_list2 = de.DataframeExtractor_csv(directory2, label_names = ["Summary"]).df_list

print("Dataset X values")
dataset_list2[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/weather-history-dataset/weatherHistory.csv
Dataset X values


,Formatted Date,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [6]:
print("Dataset Y values")
dataset_list2[1].head()

Dataset Y values


,Summary
0,Partly Cloudy
1,Partly Cloudy
2,Mostly Cloudy
3,Partly Cloudy
4,Mostly Cloudy


## Data Cleaning

## Visualization

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

## Feature Engineering

## Generating Models

## Evaluating Models

## Select and Fit Model